In [2]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio


In [3]:
import utilities as util

In [4]:
# Create a dataset from a csv file

csv_file = "dataClassified/data.csv"

# csv is in format: filename, label

prepend_path = "dataClassified/"



In [5]:
# Load datasets from util.Dataset

dataset_train = util.CustomDataset(csv_file, prepend_path, "train")
dataset_val = util.CustomDataset(csv_file, prepend_path, "val")
dataset_test = util.CustomDataset(csv_file, prepend_path, "test")

2025-02-22 15:29:16.733192: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-22 15:29:16.738232: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)


In [9]:
import finetuning_utils as ftu

In [ ]:
embeds, labels = ftu.preprocess_dataset(dataset_train, yamnet_model)


2025-02-22 15:31:46.837441: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
embeds_val, labels_val = ftu.preprocess_dataset(dataset_val, yamnet_model)

2025-02-22 15:32:09.970922: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
embeds_test, labels_test = ftu.preprocess_dataset(dataset_test, yamnet_model)

2025-02-22 15:32:23.116732: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [25]:
# Now, we will define a new Classifier model that will take the embeddings as input and output the class probabilities

from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Input(shape=(1024,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(util.num_classes, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# Now we will train the model

history = model.fit(embeds, labels, epochs=util.num_epochs, validation_data=(embeds_val, labels_val))



Epoch 1/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8530 - loss: 0.3938 - val_accuracy: 0.8922 - val_loss: 0.2844
Epoch 2/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9094 - loss: 0.2389 - val_accuracy: 0.9005 - val_loss: 0.2584
Epoch 3/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9202 - loss: 0.2124 - val_accuracy: 0.9025 - val_loss: 0.2614
Epoch 4/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9330 - loss: 0.1787 - val_accuracy: 0.8991 - val_loss: 0.2665
Epoch 5/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9405 - loss: 0.1576 - val_accuracy: 0.8996 - val_loss: 0.2744
Epoch 6/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9450 - loss: 0.1441 - val_accuracy: 0.9047 - val_loss: 0.2860
Epoch 7/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9550 - loss: 0.1246 - val_accuracy: 0.8931 - val_loss: 0.3191
Epoch 8/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9629 - loss: 0.1043 - val_accuracy: 0.

In [24]:
# Check the model performance on the test set

test_loss, test_acc = model.evaluate(embeds_test, labels_test, verbose=2)
print('\nTest accuracy:', test_acc)

132/132 - 0s - 1ms/step - accuracy: 0.8875 - loss: 0.5345

Test accuracy: 0.8875207304954529
